## Why Instance Registries?

Some components need configuration that can't easily be passed at instantiation time. For example, scorers often need:
- A configured `chat_target` for LLM-based scoring
- Specific prompt templates
- Other dependencies

Instance registries let initializers register fully-configured instances that are ready to use.

## Listing Available Instances

Use `get_names()` to see registered instances, or `list_metadata()` for details.

In [ ]:
from pyrit.prompt_target import OpenAIChatTarget
from pyrit.registry import ScorerRegistry
from pyrit.score import SelfAskRefusalScorer
from pyrit.setup import IN_MEMORY, initialize_pyrit_async

await initialize_pyrit_async(memory_db_type=IN_MEMORY)  # type: ignore

# Get the registry singleton
registry = ScorerRegistry.get_instance()

# Register a scorer instance for demonstration
chat_target = OpenAIChatTarget()
refusal_scorer = SelfAskRefusalScorer(chat_target=chat_target)
registry.register_instance(refusal_scorer)

# List what's available
names = registry.get_names()
print(f"Registered scorers: {names}")

Found default environment files: ['C:\\Users\\rlundeen\\.pyrit\\.env', 'C:\\Users\\rlundeen\\.pyrit\\.env.local']
Loaded environment file: C:\Users\rlundeen\.pyrit\.env
Loaded environment file: C:\Users\rlundeen\.pyrit\.env.local
Registered scorers: ['self_ask_refusal_e3c91809']


## Getting an Instance

Use `get()` to retrieve a pre-configured instance by name. The instance is ready to use immediately.

In [ ]:
# Get the first registered scorer
if names:
    scorer_name = names[0]
    scorer = registry.get(scorer_name)
    print(f"Retrieved scorer: {scorer}")
    print(f"Scorer type: {type(scorer).__name__}")

Retrieved scorer: <pyrit.score.true_false.self_ask_refusal_scorer.SelfAskRefusalScorer object at 0x000001D391A9DBE0>
Scorer type: SelfAskRefusalScorer


## Inspecting Metadata

Scorer metadata includes the scorer type and identifier for tracking.

In [ ]:
# Get metadata for all registered scorers
metadata = registry.list_metadata()
for item in metadata:
    print(f"\n{item['name']}:")
    print(f"  Class: {item['class_name']}")
    print(f"  Type: {item['scorer_type']}")
    print(f"  Description: {item['description'][:60]}...")


self_ask_refusal_e3c91809:
  Class: SelfAskRefusalScorer
  Type: true_false
  Description: A self-ask scorer that detects refusal in AI responses. This...


## Filtering

Use `list_metadata()` with keyword arguments to filter scorers by any metadata property. Filters use exact match for simple types and support combining multiple criteria with AND logic.

In [ ]:
# Filter by scorer_type (based on isinstance check against TrueFalseScorer/FloatScaleScorer)
true_false_scorers = registry.list_metadata(scorer_type="true_false")
print(f"True/False scorers: {[m['name'] for m in true_false_scorers]}")

# Filter by class_name
refusal_scorers = registry.list_metadata(class_name="SelfAskRefusalScorer")
print(f"Refusal scorers: {[m['name'] for m in refusal_scorers]}")

# Combine multiple filters (AND logic)
specific_scorers = registry.list_metadata(scorer_type="true_false", class_name="SelfAskRefusalScorer")
print(f"True/False refusal scorers: {[m['name'] for m in specific_scorers]}")

True/False scorers: ['self_ask_refusal_e3c91809']
Refusal scorers: ['self_ask_refusal_e3c91809']
True/False refusal scorers: ['self_ask_refusal_e3c91809']


## Key Difference from Class Registries

| Aspect | Class Registry | Instance Registry |
|--------|----------------|-------------------|
| Stores | Classes (Type[T]) | Instances (T) |
| Registration | Automatic discovery | Explicit via `register_instance()` |
| Returns | Class to instantiate | Ready-to-use instance |
| Use case | User provides config | Pre-configured by initializer |